# Inferencia en SLEAP

Este notebook realiza el análisis de video utilizando DeepLabCut para el seguimiento de movimiento animal.

## Pasos del proceso:
1. Importación de librerías necesarias
2. Carga y verificación del archivo de configuración
3. Análisis de propiedades del video
4. Recorte del video a una región de interés (ROI)
5. Análisis del video con DeepLabCut
6. Filtrado de predicciones
7. Creación de video con etiquetas

## 1. Importación de librerías

Importamos las librerías necesarias para el procesamiento de video y análisis con DeepLabCut.

In [ ]:
import deeplabcut as dlc
import cv2
import numpy as np
import os
import sys
import yaml
import matplotlib.pyplot as plt
import time

print('✓ Importación de módulos completada')

## 2. Carga del archivo de configuración

Cargamos el archivo de configuración de SLEAP/DeepLabCut y verificamos que todos los directorios necesarios existan.

In [ ]:
# Ruta al archivo de configuración
path_config_file = 'Experimento Olfato-JJ-2024-02-19/config2.yaml'

# Verificamos que la configuración exista antes de continuar
if not os.path.exists(path_config_file):
    print(f'❌ No existe el archivo de configuración en {path_config_file}')
else:
    print(f'✓ Cargando archivo {path_config_file}')
    
    # Leemos el archivo de configuración
    with open(path_config_file) as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
    print('\nContenido de la configuración:')
    print(config)

    # Verificamos el 'project_path' ya que es esencial que esté configurado con la ubicación correcta
    if not os.path.exists(config['project_path']):
        print(f'\n❌ No existe el directorio {config["project_path"]} configurado en {path_config_file}')
    else:
        print(f'\n✓ El directorio del proyecto {config["project_path"]} existe')

## 3. Análisis de propiedades del video

Cargamos el video y extraemos sus propiedades principales (fps, número de frames, dimensiones, etc.).
También visualizamos un frame aleatorio para verificar que el video se cargó correctamente.

In [ ]:
# Ruta al video de entrada
video_path = 'experiments/A-7.mp4'

# Verificamos que el video exista
if not os.path.exists(video_path):
    print(f'❌ No existe el archivo de video en {video_path}')
    raise SystemExit("El video no fue encontrado. Verifique la ruta.")
else:
    print(f'✓ Cargando video: {video_path}')

# Abrimos el video y extraemos sus propiedades
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()

# Mostramos las propiedades del video
print(f'\nPropiedades del video:')
print(f'  Frame leído correctamente: {ret}')
print(f'  FPS (cuadros por segundo): {cap.get(cv2.CAP_PROP_FPS)}')
print(f'  Número total de frames: {int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}')
print(f'  Ancho: {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))} píxeles')
print(f'  Alto: {int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))} píxeles')
print(f'  Codec (FOURCC): {int(cap.get(cv2.CAP_PROP_FOURCC))}')

# Extraemos y mostramos un frame aleatorio
random_frame = np.random.randint(0, int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
cap.set(cv2.CAP_PROP_POS_FRAMES, random_frame)
ret, frame = cap.read()

plt.figure(figsize=(10, 6))
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title(f'Frame aleatorio #{random_frame}')
plt.tight_layout()
plt.show()

cap.release()

## 4. Recorte del video a una región de interés (ROI)

Recortamos el video a una región de interés específica para enfocarnos en el área relevante del experimento.
Esto reduce el tiempo de procesamiento y mejora la precisión del análisis.

In [ ]:
# Definimos la región de interés (ROI): (x, y, ancho, alto)
roi_x, roi_y, roi_w, roi_h = 30, 70, 520, 245

# Rutas de entrada y salida
video_path = 'experiments/A-7.mp4'
video_out = 'experiments/A-7_crop.mp4'

# Abrimos el video de entrada
cap = cv2.VideoCapture(video_path)

# Obtenemos las propiedades del video
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Configuramos el codec y creamos el objeto VideoWriter para el video de salida
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_out, fourcc, fps, (roi_w, roi_h))

print(f'Procesando {frame_count} frames...')

# Procesamos cada frame del video
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Recortamos el frame a la ROI definida
    roi_frame = frame[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]
    
    # Escribimos el frame recortado en el video de salida
    out.write(roi_frame)
    frame_num += 1

# Liberamos los recursos
cap.release()
out.release()

print(f'✓ Video recortado guardado en: {video_out}')
print(f'  Frames procesados: {frame_num}')

## 5. Análisis del video con DeepLabCut

Ejecutamos el análisis del video recortado utilizando DeepLabCut para detectar y rastrear los puntos clave del animal.

In [ ]:
# Video a analizar
video_anal = 'experiments/A-7_crop.mp4'

print(f'Iniciando análisis de {video_anal}...')
start_time = time.time()

# Ejecutamos el análisis con DeepLabCut
dlc.analyze_videos(
    path_config_file,
    [video_anal],
    videotype='mp4',
    shuffle=1,
    trainingsetindex=0,
    gputouse=None,
    save_as_csv=True
)

elapsed_time = time.time() - start_time
print(f'\n✓ Análisis completado en {elapsed_time:.2f} segundos ({elapsed_time/60:.2f} minutos)')

## 6. Filtrado de predicciones

Aplicamos un filtro ARIMA a las predicciones para suavizar las trayectorias y reducir el ruido en los datos de seguimiento.

In [ ]:
print('Aplicando filtrado ARIMA a las predicciones...')

dlc.filterpredictions(
    path_config_file,
    [video_anal],
    shuffle=1,
    trainingsetindex=0,
    filtertype='arima',
    windowlength=5,
    p_bound=0.7,
    ARdegree=3,
    MAdegree=1,
    alpha=0.01,
    save_as_csv=True,
    destfolder=None,
    modelprefix='',
    track_method='',
    return_data=False
)

print('✓ Filtrado completado')

## 7. Creación de video con etiquetas

Generamos un video con las predicciones superpuestas para visualizar los resultados del seguimiento.

**Nota:** Ajuste la ruta `output_video` según su sistema antes de ejecutar esta celda.

In [ ]:
# Ruta al video que se desea etiquetar
# NOTA: Ajuste esta ruta según la ubicación de su video
output_video = 'experiments/A-7_crop.mp4'  # Cambie por la ruta deseada

print(f'Creando video etiquetado para: {output_video}')

dlc.create_labeled_video(
    path_config_file,
    [output_video],
    shuffle=1,
    trainingsetindex=0,
    filtered=False,
    save_frames=False,
    displayedbodyparts='all',
    draw_skeleton=False,
    trailpoints=0,
    videotype='mp4'
)

print('✓ Video etiquetado creado exitosamente')